# MUC16 Mutation Trans Effect on Transcriptomics

This notebook analyzes the trans effect of MUC16 mutation on interacting and other proteins Transcriptomics, in Endometrial, Colon, and Ovarian cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot
import matplotlib.pyplot as plt
import seaborn as sns

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()
ov = cptac.Ovarian()

### Investigate KRAS gene

In [2]:
gene = "MUC16"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

# Interacting Proteins: Transcriptomics

## Endometrial

### Generate interacting protein list

In [4]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
ST6GAL1
ST6GALNAC2
C1GALT1C1
ST6GALNAC3
ST3GAL2
MUC16
MSLN
C1GALT1
MUC6
ST3GAL1
MUC5B
MUC1
MUC13
MUC5AC
LGALS1
IGBP1
RAD21
MCM2
EGFR
TRIM25


### Test for significant comparisons in any of interacting proteins

In [6]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons

             Comparison   P_Value
0  MCM2_transcriptomics  0.002101





## Colon

### Generate interacting protein list

In [7]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = co.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
ST6GAL1
C1GALT1C1
MSLN
C1GALT1
MUC6
MUC4
ST3GAL4
MUC5B
MUC1
MUC13
MUC5AC
LGALS1
IGBP1
RAD21
MCM2
EGFR
TRIM25
KIAA1429


### Test for significant comparisons in any of interacting proteins

In [8]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

MUC6 did not match any columns in transcriptomics dataframe. MUC6_transcriptomics column inserted, but filled with NaN.
MUC5B did not match any columns in transcriptomics dataframe. MUC5B_transcriptomics column inserted, but filled with NaN.
MUC5AC did not match any columns in transcriptomics dataframe. MUC5AC_transcriptomics column inserted, but filled with NaN.
Doing t-test comparisons

              Comparison   P_Value
0  IGBP1_transcriptomics  0.002351





## Ovarian

### Generate interacting protein list

In [9]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = ov.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
ST6GAL1
C1GALT1C1
MUC16
MSLN
C1GALT1
MUC4
ST3GAL1
ST3GAL4
MUC5B
MUC1
LGALS1
IGBP1
RAD21
MCM2
EGFR
TRIM25


In [10]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

C1GALT1 did not match any columns in transcriptomics dataframe. C1GALT1_transcriptomics column inserted, but filled with NaN.
MUC1 did not match any columns in transcriptomics dataframe. MUC1_transcriptomics column inserted, but filled with NaN.
LGALS1 did not match any columns in transcriptomics dataframe. LGALS1_transcriptomics column inserted, but filled with NaN.
RAD21 did not match any columns in transcriptomics dataframe. RAD21_transcriptomics column inserted, but filled with NaN.
TRIM25 did not match any columns in transcriptomics dataframe. TRIM25_transcriptomics column inserted, but filled with NaN.
Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics

## Endometrial

In [11]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  MUC16
MT-ATP6 did not match any columns in transcriptomics dataframe. MT-ATP6_transcriptomics column inserted, but filled with NaN.
MT-ATP8 did not match any columns in transcriptomics dataframe. MT-ATP8_transcriptomics column inserted, but filled with NaN.
MT-CO1 did not match any columns in transcriptomics dataframe. MT-CO1_transcriptomics column inserted, but filled with NaN.
MT-CO2 did not match any columns in transcriptomics dataframe. MT-CO2_transcriptomics column inserted, but filled with NaN.
MT-CO3 did not match any columns in transcriptomics dataframe. MT-CO3_transcriptomics column inserted, but filled with NaN.
MT-CYB did not match any columns in transcriptomics dataframe. MT-CYB_transcriptomics column inserted, but filled with NaN.
MT-ND1 did not match any columns in transcriptomics dataframe. MT-ND1_transcriptomics column inserted, but filled with NaN.
MT-ND2 did not match any columns in transcriptomics dataframe. MT-ND2_transcriptomics column inserted, but filled 

## Colon

In [12]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = co.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  MUC16
A1BG did not match any columns in transcriptomics dataframe. A1BG_transcriptomics column inserted, but filled with NaN.
ABCA8 did not match any columns in transcriptomics dataframe. ABCA8_transcriptomics column inserted, but filled with NaN.
ACAN did not match any columns in transcriptomics dataframe. ACAN_transcriptomics column inserted, but filled with NaN.
ACTA1 did not match any columns in transcriptomics dataframe. ACTA1_transcriptomics column inserted, but filled with NaN.
ACTBL2 did not match any columns in transcriptomics dataframe. ACTBL2_transcriptomics column inserted, but filled with NaN.
ACTL8 did not match any columns in transcriptomics dataframe. ACTL8_transcriptomics column inserted, but filled with NaN.
ACTN2 did not match any columns in transcriptomics dataframe. ACTN2_transcriptomics column inserted, but filled with NaN.
ACTN3 did not match any columns in transcriptomics dataframe. ACTN3_transcriptomics column inserted, but filled with NaN.
ADAMTS16 did

CA7 did not match any columns in transcriptomics dataframe. CA7_transcriptomics column inserted, but filled with NaN.
CADM2 did not match any columns in transcriptomics dataframe. CADM2_transcriptomics column inserted, but filled with NaN.
CADM3 did not match any columns in transcriptomics dataframe. CADM3_transcriptomics column inserted, but filled with NaN.
CALB1 did not match any columns in transcriptomics dataframe. CALB1_transcriptomics column inserted, but filled with NaN.
CALML3 did not match any columns in transcriptomics dataframe. CALML3_transcriptomics column inserted, but filled with NaN.
CALML5 did not match any columns in transcriptomics dataframe. CALML5_transcriptomics column inserted, but filled with NaN.
CALR3 did not match any columns in transcriptomics dataframe. CALR3_transcriptomics column inserted, but filled with NaN.
CAMK2B did not match any columns in transcriptomics dataframe. CAMK2B_transcriptomics column inserted, but filled with NaN.
CAMK4 did not match an

CTNNA3 did not match any columns in transcriptomics dataframe. CTNNA3_transcriptomics column inserted, but filled with NaN.
CTRB2 did not match any columns in transcriptomics dataframe. CTRB2_transcriptomics column inserted, but filled with NaN.
CTRC did not match any columns in transcriptomics dataframe. CTRC_transcriptomics column inserted, but filled with NaN.
CYLC2 did not match any columns in transcriptomics dataframe. CYLC2_transcriptomics column inserted, but filled with NaN.
CYP3A4 did not match any columns in transcriptomics dataframe. CYP3A4_transcriptomics column inserted, but filled with NaN.
CYP4F8 did not match any columns in transcriptomics dataframe. CYP4F8_transcriptomics column inserted, but filled with NaN.
CYS1 did not match any columns in transcriptomics dataframe. CYS1_transcriptomics column inserted, but filled with NaN.
DAPL1 did not match any columns in transcriptomics dataframe. DAPL1_transcriptomics column inserted, but filled with NaN.
DCLK1 did not match an

GNG7 did not match any columns in transcriptomics dataframe. GNG7_transcriptomics column inserted, but filled with NaN.
GNRH2 did not match any columns in transcriptomics dataframe. GNRH2_transcriptomics column inserted, but filled with NaN.
GP1BB did not match any columns in transcriptomics dataframe. GP1BB_transcriptomics column inserted, but filled with NaN.
GP2 did not match any columns in transcriptomics dataframe. GP2_transcriptomics column inserted, but filled with NaN.
GP9 did not match any columns in transcriptomics dataframe. GP9_transcriptomics column inserted, but filled with NaN.
GPX6 did not match any columns in transcriptomics dataframe. GPX6_transcriptomics column inserted, but filled with NaN.
GREM2 did not match any columns in transcriptomics dataframe. GREM2_transcriptomics column inserted, but filled with NaN.
GSDMA did not match any columns in transcriptomics dataframe. GSDMA_transcriptomics column inserted, but filled with NaN.
GSTT2 did not match any columns in t

KRT14 did not match any columns in transcriptomics dataframe. KRT14_transcriptomics column inserted, but filled with NaN.
KRT16 did not match any columns in transcriptomics dataframe. KRT16_transcriptomics column inserted, but filled with NaN.
KRT2 did not match any columns in transcriptomics dataframe. KRT2_transcriptomics column inserted, but filled with NaN.
KRT3 did not match any columns in transcriptomics dataframe. KRT3_transcriptomics column inserted, but filled with NaN.
KRT31 did not match any columns in transcriptomics dataframe. KRT31_transcriptomics column inserted, but filled with NaN.
KRT34 did not match any columns in transcriptomics dataframe. KRT34_transcriptomics column inserted, but filled with NaN.
KRT36 did not match any columns in transcriptomics dataframe. KRT36_transcriptomics column inserted, but filled with NaN.
KRT5 did not match any columns in transcriptomics dataframe. KRT5_transcriptomics column inserted, but filled with NaN.
KRT6A did not match any column

NOS1 did not match any columns in transcriptomics dataframe. NOS1_transcriptomics column inserted, but filled with NaN.
NOS1AP did not match any columns in transcriptomics dataframe. NOS1AP_transcriptomics column inserted, but filled with NaN.
NOVA2 did not match any columns in transcriptomics dataframe. NOVA2_transcriptomics column inserted, but filled with NaN.
NPY did not match any columns in transcriptomics dataframe. NPY_transcriptomics column inserted, but filled with NaN.
NTN1 did not match any columns in transcriptomics dataframe. NTN1_transcriptomics column inserted, but filled with NaN.
OGDHL did not match any columns in transcriptomics dataframe. OGDHL_transcriptomics column inserted, but filled with NaN.
ORM1 did not match any columns in transcriptomics dataframe. ORM1_transcriptomics column inserted, but filled with NaN.
ORM2 did not match any columns in transcriptomics dataframe. ORM2_transcriptomics column inserted, but filled with NaN.
OTC did not match any columns in t

RPL3L did not match any columns in transcriptomics dataframe. RPL3L_transcriptomics column inserted, but filled with NaN.
RPS4Y1 did not match any columns in transcriptomics dataframe. RPS4Y1_transcriptomics column inserted, but filled with NaN.
RPS4Y2 did not match any columns in transcriptomics dataframe. RPS4Y2_transcriptomics column inserted, but filled with NaN.
RTN1 did not match any columns in transcriptomics dataframe. RTN1_transcriptomics column inserted, but filled with NaN.
S100A7 did not match any columns in transcriptomics dataframe. S100A7_transcriptomics column inserted, but filled with NaN.
SAA2-SAA4 did not match any columns in transcriptomics dataframe. SAA2-SAA4_transcriptomics column inserted, but filled with NaN.
SARDH did not match any columns in transcriptomics dataframe. SARDH_transcriptomics column inserted, but filled with NaN.
SCGN did not match any columns in transcriptomics dataframe. SCGN_transcriptomics column inserted, but filled with NaN.
SCN7A did not 

TMEM189-UBE2V1 did not match any columns in transcriptomics dataframe. TMEM189-UBE2V1_transcriptomics column inserted, but filled with NaN.
TMEM35A did not match any columns in transcriptomics dataframe. TMEM35A_transcriptomics column inserted, but filled with NaN.
TMOD1 did not match any columns in transcriptomics dataframe. TMOD1_transcriptomics column inserted, but filled with NaN.
TMPPE did not match any columns in transcriptomics dataframe. TMPPE_transcriptomics column inserted, but filled with NaN.
TNXB did not match any columns in transcriptomics dataframe. TNXB_transcriptomics column inserted, but filled with NaN.
TPPP did not match any columns in transcriptomics dataframe. TPPP_transcriptomics column inserted, but filled with NaN.
TRIM72 did not match any columns in transcriptomics dataframe. TRIM72_transcriptomics column inserted, but filled with NaN.
TTR did not match any columns in transcriptomics dataframe. TTR_transcriptomics column inserted, but filled with NaN.
TUBA3C d

## Ovarian

In [17]:

print("\nGene: ", gene)

'''Use all proteins'''
proteomics = ov.get_proteomics()
all_proteins = list(set(proteomics.columns))

'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
protdf = protdf.loc[:,~protdf.columns.duplicated()]

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the datafram correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")



Gene:  MUC16
CCL14 did not match any columns in transcriptomics dataframe. CCL14_transcriptomics column inserted, but filled with NaN.
SULT1A4 did not match any columns in transcriptomics dataframe. SULT1A4_transcriptomics column inserted, but filled with NaN.
DUS3L did not match any columns in transcriptomics dataframe. DUS3L_transcriptomics column inserted, but filled with NaN.
NBR1 did not match any columns in transcriptomics dataframe. NBR1_transcriptomics column inserted, but filled with NaN.
DDT did not match any columns in transcriptomics dataframe. DDT_transcriptomics column inserted, but filled with NaN.
B3GLCT did not match any columns in transcriptomics dataframe. B3GLCT_transcriptomics column inserted, but filled with NaN.
S100A4 did not match any columns in transcriptomics dataframe. S100A4_transcriptomics column inserted, but filled with NaN.
SGSM3 did not match any columns in transcriptomics dataframe. SGSM3_transcriptomics column inserted, but filled with NaN.
MYCBP di

NSMCE3 did not match any columns in transcriptomics dataframe. NSMCE3_transcriptomics column inserted, but filled with NaN.
R3HDM2 did not match any columns in transcriptomics dataframe. R3HDM2_transcriptomics column inserted, but filled with NaN.
PHKG2 did not match any columns in transcriptomics dataframe. PHKG2_transcriptomics column inserted, but filled with NaN.
VMAC did not match any columns in transcriptomics dataframe. VMAC_transcriptomics column inserted, but filled with NaN.
MAP3K21 did not match any columns in transcriptomics dataframe. MAP3K21_transcriptomics column inserted, but filled with NaN.
RPS11 did not match any columns in transcriptomics dataframe. RPS11_transcriptomics column inserted, but filled with NaN.
RIOX1 did not match any columns in transcriptomics dataframe. RIOX1_transcriptomics column inserted, but filled with NaN.
FAM89B did not match any columns in transcriptomics dataframe. FAM89B_transcriptomics column inserted, but filled with NaN.
GON7 did not mat

GLOD4 did not match any columns in transcriptomics dataframe. GLOD4_transcriptomics column inserted, but filled with NaN.
DGAT1 did not match any columns in transcriptomics dataframe. DGAT1_transcriptomics column inserted, but filled with NaN.
ZNF250 did not match any columns in transcriptomics dataframe. ZNF250_transcriptomics column inserted, but filled with NaN.
ZNF592 did not match any columns in transcriptomics dataframe. ZNF592_transcriptomics column inserted, but filled with NaN.
BICRA did not match any columns in transcriptomics dataframe. BICRA_transcriptomics column inserted, but filled with NaN.
ARMT1 did not match any columns in transcriptomics dataframe. ARMT1_transcriptomics column inserted, but filled with NaN.
AMT did not match any columns in transcriptomics dataframe. AMT_transcriptomics column inserted, but filled with NaN.
GPRASP2 did not match any columns in transcriptomics dataframe. GPRASP2_transcriptomics column inserted, but filled with NaN.
TMEM219 did not matc

NPEPL1 did not match any columns in transcriptomics dataframe. NPEPL1_transcriptomics column inserted, but filled with NaN.
ESS2 did not match any columns in transcriptomics dataframe. ESS2_transcriptomics column inserted, but filled with NaN.
NXN did not match any columns in transcriptomics dataframe. NXN_transcriptomics column inserted, but filled with NaN.
RPL13A did not match any columns in transcriptomics dataframe. RPL13A_transcriptomics column inserted, but filled with NaN.
EIF4A2 did not match any columns in transcriptomics dataframe. EIF4A2_transcriptomics column inserted, but filled with NaN.
MRNIP did not match any columns in transcriptomics dataframe. MRNIP_transcriptomics column inserted, but filled with NaN.
PXMP4 did not match any columns in transcriptomics dataframe. PXMP4_transcriptomics column inserted, but filled with NaN.
MAPK3 did not match any columns in transcriptomics dataframe. MAPK3_transcriptomics column inserted, but filled with NaN.
FAM25G did not match any

PRIM1 did not match any columns in transcriptomics dataframe. PRIM1_transcriptomics column inserted, but filled with NaN.
SIRT7 did not match any columns in transcriptomics dataframe. SIRT7_transcriptomics column inserted, but filled with NaN.
RPL38 did not match any columns in transcriptomics dataframe. RPL38_transcriptomics column inserted, but filled with NaN.
SCAMP4 did not match any columns in transcriptomics dataframe. SCAMP4_transcriptomics column inserted, but filled with NaN.
CA5B did not match any columns in transcriptomics dataframe. CA5B_transcriptomics column inserted, but filled with NaN.
NDRG2 did not match any columns in transcriptomics dataframe. NDRG2_transcriptomics column inserted, but filled with NaN.
NECTIN2 did not match any columns in transcriptomics dataframe. NECTIN2_transcriptomics column inserted, but filled with NaN.
MOB4 did not match any columns in transcriptomics dataframe. MOB4_transcriptomics column inserted, but filled with NaN.
ERI3 did not match any

...

ZMAT2 did not match any columns in transcriptomics dataframe. ZMAT2_transcriptomics column inserted, but filled with NaN.
ARHGEF35 did not match any columns in transcriptomics dataframe. ARHGEF35_transcriptomics column inserted, but filled with NaN.
RAB24 did not match any columns in transcriptomics dataframe. RAB24_transcriptomics column inserted, but filled with NaN.
CORO1A did not match any columns in transcriptomics dataframe. CORO1A_transcriptomics column inserted, but filled with NaN.
COL7A1 did not match any columns in transcriptomics dataframe. COL7A1_transcriptomics column inserted, but filled with NaN.
CAPN7 did not match any columns in transcriptomics dataframe. CAPN7_transcriptomics column inserted, but filled with NaN.
PTBP3 did not match any columns in transcriptomics dataframe. PTBP3_transcriptomics column inserted, but filled with NaN.
CHKB did not match any columns in transcriptomics dataframe. CHKB_transcriptomics column inserted, but filled with NaN.
ARPC4 did not ma

DCTN1 did not match any columns in transcriptomics dataframe. DCTN1_transcriptomics column inserted, but filled with NaN.
FKBP5 did not match any columns in transcriptomics dataframe. FKBP5_transcriptomics column inserted, but filled with NaN.
SMN1 did not match any columns in transcriptomics dataframe. SMN1_transcriptomics column inserted, but filled with NaN.
KIF1BP did not match any columns in transcriptomics dataframe. KIF1BP_transcriptomics column inserted, but filled with NaN.
POM121C did not match any columns in transcriptomics dataframe. POM121C_transcriptomics column inserted, but filled with NaN.
PLAUR did not match any columns in transcriptomics dataframe. PLAUR_transcriptomics column inserted, but filled with NaN.
NECTIN1 did not match any columns in transcriptomics dataframe. NECTIN1_transcriptomics column inserted, but filled with NaN.
CD27 did not match any columns in transcriptomics dataframe. CD27_transcriptomics column inserted, but filled with NaN.
COQ8B did not matc

KRTDAP did not match any columns in transcriptomics dataframe. KRTDAP_transcriptomics column inserted, but filled with NaN.
LRMDA did not match any columns in transcriptomics dataframe. LRMDA_transcriptomics column inserted, but filled with NaN.
SELENOT did not match any columns in transcriptomics dataframe. SELENOT_transcriptomics column inserted, but filled with NaN.
LMAN2 did not match any columns in transcriptomics dataframe. LMAN2_transcriptomics column inserted, but filled with NaN.
PHC2 did not match any columns in transcriptomics dataframe. PHC2_transcriptomics column inserted, but filled with NaN.
VPS45 did not match any columns in transcriptomics dataframe. VPS45_transcriptomics column inserted, but filled with NaN.
RBMY1E did not match any columns in transcriptomics dataframe. RBMY1E_transcriptomics column inserted, but filled with NaN.
RAD21 did not match any columns in transcriptomics dataframe. RAD21_transcriptomics column inserted, but filled with NaN.
NBL1 did not match

UBL4A did not match any columns in transcriptomics dataframe. UBL4A_transcriptomics column inserted, but filled with NaN.
UQCC3 did not match any columns in transcriptomics dataframe. UQCC3_transcriptomics column inserted, but filled with NaN.
BRD1 did not match any columns in transcriptomics dataframe. BRD1_transcriptomics column inserted, but filled with NaN.
MRPS17 did not match any columns in transcriptomics dataframe. MRPS17_transcriptomics column inserted, but filled with NaN.
CTPS1 did not match any columns in transcriptomics dataframe. CTPS1_transcriptomics column inserted, but filled with NaN.
UFD1 did not match any columns in transcriptomics dataframe. UFD1_transcriptomics column inserted, but filled with NaN.
NAXE did not match any columns in transcriptomics dataframe. NAXE_transcriptomics column inserted, but filled with NaN.
VIPAS39 did not match any columns in transcriptomics dataframe. VIPAS39_transcriptomics column inserted, but filled with NaN.
LRRFIP1 did not match an

C1QC did not match any columns in transcriptomics dataframe. C1QC_transcriptomics column inserted, but filled with NaN.
CDK11A did not match any columns in transcriptomics dataframe. CDK11A_transcriptomics column inserted, but filled with NaN.
JTB did not match any columns in transcriptomics dataframe. JTB_transcriptomics column inserted, but filled with NaN.
HIGD1A did not match any columns in transcriptomics dataframe. HIGD1A_transcriptomics column inserted, but filled with NaN.
PAN2 did not match any columns in transcriptomics dataframe. PAN2_transcriptomics column inserted, but filled with NaN.
TXNDC9 did not match any columns in transcriptomics dataframe. TXNDC9_transcriptomics column inserted, but filled with NaN.
MRPL30 did not match any columns in transcriptomics dataframe. MRPL30_transcriptomics column inserted, but filled with NaN.
MED22 did not match any columns in transcriptomics dataframe. MED22_transcriptomics column inserted, but filled with NaN.
RDH14 did not match any 

## Conclusions

MCM2 showed significantly altered transcriptomics when MUC16 was mutated in Endometrial cancer. It is a known interacting partner to MUC16. IGBP1 showed significantly altered transcriptomics when MUC16 was mutated in Colon cancer. It is also a known interacting partner to MUC16. NUP188, SFXN, SLC37A4, PSMB6, and UQCRC1 also all showed significantly altered transcriptomics when MUC17 was mutated in Endometrial cancer. TYMS, PXMP4, and ERGIC3 also all showed significantly altered transcriptomics when MUC17 was mutated in Colon Cancer.